# Music Theory

TO DO:

    a) allow for more extension (i.e 9, 13...) in chord2notes
    b) note string notation parser (quality and extension together?)
    c) arpeggio for 3 and 4

In [1]:
### Chromatic Scale: 12-tone of equal semitone temperament
CHROMATIC = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
## Heptatonic Scales: 7-tone
SCALE_WEIGHTS = [0.5,0,0,0,0,0.5,0,0,0]
SCALE_INTERVALS = {
    # Diatonic Modes: 5 tones and 2 semitones, distance between semitones greater than 2 tones
    'major':      [2,2,1,2,2,2,1], # I (or ionian)
    'dorian':     [2,1,2,2,2,1,2], # II
    'phrygian':   [1,2,2,2,1,2,2], # III
    'lydian':     [2,2,2,1,2,2,1], # IV
    'mixolydian': [2,2,1,2,2,1,2], # V
    'minor':      [2,1,2,2,1,2,2], # VI (or aeolian)
    'locrian':    [1,2,2,1,2,2,2], # VII
    # Other heptatonics:
    'melodic':    [2,1,2,2,2,2,1],
    'harmonic':   [2,1,2,2,1,3,1]
}

def scale_generator(rootNote, scale = 'major'):
    interval, root = SCALE_INTERVALS[scale], CHROMATIC.index(rootNote)
    scaleIndexes = [(root + sum(interval[:i]))%12 for i in range(7)]
    scaleNotes = [CHROMATIC[i] for i in scaleIndexes]
    return scaleNotes

In [2]:
scale_generator('B', 'locrian')

['B', 'C', 'D', 'E', 'F', 'G', 'A']

In [3]:
BASE_CHORD_INTERVAL = [0, 4, 7]
TRIAD_INTERVALS = {
    '':  [0,0,0],
    'm': [0,-1,0],
    '+': [0,0,1],
    'o': [0,-1,-1]
}

EXTENSION_WEIGHTS = [0.9, 0.05, 0.05]
EXTENSION_INTERVALS = {
    '':     None,
    '7':    3,
    'maj7': 4
}

class Chord:
    def __init__(self, root, triad = '', extension = '', octave = 4):
        self.root = root
        self.triad = triad
        self.extension = extension
        self.octave = octave
        self.length = 3 if extension == '' else 4
    
    def __repr__(self):
        return self.root + self.triad + self.extension
    
    def indexes(self):
        root = CHROMATIC.index(self.root)
        triad = TRIAD_INTERVALS[self.triad]
        extension = EXTENSION_INTERVALS[self.extension]
        chordIndexes = [(root + triad[i] + BASE_CHORD_INTERVAL[i])%12 for i in range(3)]
        if extension != None:
            chordIndexes = chordIndexes + [(chordIndexes[-1] + extension)%12]
        return chordIndexes
    
    def notes(self):
        chordIndexes = self.indexes()
        chordNotes = [CHROMATIC[i] for i in chordIndexes]
        return chordNotes
    
    def octaves(self):
        chordIndexes = self.indexes()
        chordOctaves = [self.octave]
        for i in range(len(chordIndexes)-1):
            if chordIndexes[i+1] < chordIndexes[i]:
                chordOctaves.append(self.octave + 1)
            else:
                chordOctaves.append(self.octave)
        return chordOctaves

In [4]:
c = Chord('D', 'm', '7')

In [5]:
c

Dm7

In [6]:
c.indexes()

[2, 5, 9, 0]

In [7]:
c.notes()

['D', 'F', 'A', 'C']

In [8]:
c.octaves()

[4, 4, 4, 5]

In [9]:
def notes2chord(notes):
    notesIndexes = [CHROMATIC.index(n) for n in notes]
    # Correcting Octaves
    for i in range(2):
        if notesIndexes[i+1] < notesIndexes[i]:
            notesIndexes[i+1] += 12
    triadIntervals = [notesIndexes[i] - BASE_CHORD_INTERVAL[i] - notesIndexes[0] for i in range(3)]
    for triad in TRIAD_INTERVALS: # finding a match for the triad quality
        if TRIAD_INTERVALS[triad] == triadIntervals:
            return Chord(notes[0], triad)
    else:
        print('Chord not found.')
        return None

def harmonic_field(scale):
    I_III_V_rule = [[scale[i], scale[(i+2)%7], scale[(i+4)%7]] for i in range(7)]
    harmonicField = [notes2chord(notes) for notes in I_III_V_rule]
    return harmonicField

In [10]:
scale = scale_generator('C', 'minor')
hf = harmonic_field(scale)
hf

[Cm, Do, D#, Fm, Gm, G#, A#]

# Music Generation

In [11]:
import random

## Harmony

In [12]:
def chord_progression(harmonicField, size, I_V_rule = True, addExtensions = False):
    if I_V_rule:
        CP = [0] + [random.randint(0,6) for _ in range(size - 2)] + [4]
    else:
        CP = [random.randint(0,6) for _ in range(size)]
    roots = [harmonicField[i].root for i in CP]
    triads = [harmonicField[i].triad for i in CP]
    if addExtensions:
        extensions = random.choices(list(EXTENSION_INTERVALS), EXTENSION_WEIGHTS, k = size)
        chords = zip(roots, triads, extensions)
    else:
        chords = zip(roots, triads)
    return [Chord(*args) for args in chords]

In [13]:
hf = harmonic_field(scale_generator('B', 'major'))
chord_progression(hf, 4)

[B, G#m, G#m, F#]

In [14]:
def arpeggio_generator(T, numberNotes, numberDivisions):
    times = random.sample([i for i in range(1, numberDivisions)], k = numberNotes - 1)
    times.sort()
    times = [T*t/numberDivisions for t in times]
    return [times[0]] + [times[i+1] - times[i] for i in range(numberNotes - 2)] + [T - times[-1]]

In [15]:
a = arpeggio_generator(1.8, 3, 12)
print(a)
sum(a)

[0.15, 0.75, 0.9]


1.8

## Melody

In [16]:
scale = scale_generator('C', 'minor')
scale

['C', 'D', 'D#', 'F', 'G', 'G#', 'A#']

## Rhythm

In [17]:
kick = 42
hh = 37
snare = 47

drums = [
    [
        {'note':     [hh, kick, hh, hh, snare, hh],
         'velocity': [80, 112, 80, 80, 80, 100, 80],
         'time':     [0, 0.5, 0.4, 0, 0.5, 0.4]},
        {'note':     [hh, kick, hh, hh, snare, hh, kick],
         'velocity': [80, 112, 80, 80, 80, 100, 80, 112],
         'time':     [0, 0.5, 0.4, 0, 0.5, 0.25, 0.15]}
    ], [
        {'note':     [hh, kick, hh, hh, snare, hh, kick],
         'velocity': [80, 112, 80, 80, 100, 80, 80, 80],
         'time':     [0, 0.45, 0.45, 0, 0.4, 0.05, 0.4, 0.05]},
        {'note':     [hh, hh, kick, hh, snare, kick, kick, hh],
         'velocity': [80, 80, 80, 80, 100, 80, 80, 80],
         'time':     [0.45, 0, 0.45, 0, 0.3, 0.15, 0, 0.45]}
    ], [
        {'note':     [hh, kick, 40, hh, hh, snare, kick, hh],
         'velocity': [80, 112, 80, 80, 80, 100, 80, 80],
         'time':     [0, 0.225, 0.225, 0.45, 0, 0.4, 0.05, 0.45]},
        {'note':     [hh, hh, kick, hh, snare, kick, hh],
         'velocity': [80, 80, 80, 80, 100, 80, 80],
         'time':     [0.45, 0.35, 0.10, 0, 0.4, 0.05, 0.45]}
    ], [
        {'note':     [hh, kick, hh, hh, snare, hh],
         'velocity': [80, 112, 80, 80, 100, 80],
         'time':     [0, 0.55, 0.35, 0, 0.55, 0.35]},
        {'note':     [hh, kick, hh, hh, snare, kick, kick, hh],
         'velocity': [80, 112, 80, 80, 100, 112, 112, 80],
         'time':     [0, 0.55, 0.35, 0, 0.15, 0.15, 0.25, 0.35]}
    ], [
        {'note':     [0, hh, hh, hh, snare, hh, kick],
         'velocity': [80, 80, 80, 80, 100, 80, 100],
         'time':     [0.5, 0.05, 0.05, 0.3, 0.5, 0.3, 0.1]},
        {'note':     [kick, hh, snare, kick, hh],
         'velocity': [80, 100, 80, 100, 100],
         'time':     [0.5, 0.4, 0.4, 0.1, 0.4]}
    ]
]

def playDrumsprimitivo(bateria, drumflag):
    pattern = drums[bateria][drumflag]
    for note, velocity, wait in zip(pattern['note'], pattern['velocity'], pattern['time']):
        midiout.send_message([0x90, note, velocity])
        time.sleep(wait)

# MIDI

In [18]:
import mido

In [19]:
import rtmidi

midiout = rtmidi.MidiOut()
available_ports = midiout.get_ports()

# here we're printing the ports to check that we see the one that loopMidi created. 
# In the list we should see a port called "loopMIDI port".
print(available_ports)

# Attempt to open the port
if available_ports:
    midiout.open_port(1)
else:
    midiout.open_virtual_port("My virtual output")

['Microsoft GS Wavetable Synth 0', 'loopMIDI Port 1', 'loopMIDI Port 1 2', 'loopMIDI Port 2 3']


In [5]:
# PORT SETUP
def port_setup():
    portNames = mido.get_output_names()
    port = mido.open_output(portNames[1])
    return port

In [ ]:
# UTILITY
port.close() # CAN'T OPEN THE SAME PORT TWICE. CLOSE IT FIRST.
port.reset() # RESET ALL NOTS AND CONTROLLERS
port.panic() # RESET ALL NOTES

In [ ]:
# READ/PLAY MIDI FILES
mid = mido.MidiFile('Wish_You_Were_Here_-_Pink_Floyd.mid')
try:
    for msg in mid.play():
        port.send(msg)
except KeyboardInterrupt:
    port.reset()
    pass

In [20]:
def note2MIDI(note, octave = 4):
    noteIndex = CHROMATIC.index(note)
    # Return closest octave when out of bounds
    if octave < -1:
        return noteIndex
    elif noteIndex > 7 and octave == 9 or octave > 9:
        return noteIndex + 108 if noteIndex > 7 else noteIndex + 120
    return CHROMATIC.index(note) + 12*(octave + 1)

In [21]:
note2MIDI('A', 11)

117

# Main Program

MUSIC IDEAS:

    a) add conversation at the intro before beat starts
    b) place melody in higher octaves (improve perception)
    c) place mini delays in chords to sound more human

In [29]:
#port = port_setup()
import threading
import time
T = 1.8
PROGRESSION_SIZE = 4
ARPEGGIO_DIVISIONS = 12


while True:
        
    # SCALE
    randomScale = random.choices(list(SCALE_INTERVALS), SCALE_WEIGHTS, k = 1)[0]
    randomRoot = random.choice(CHROMATIC)
    scale = scale_generator(randomRoot, randomScale)
    harmonicField = harmonic_field(scale)

    # HARMONY
    instrumentHarmony = random.randint(8, 14) + 144
    chordProgression = chord_progression(harmonicField, PROGRESSION_SIZE, I_V_rule = True, addExtensions = False)
    arpeggio = arpeggio_generator(T, chordProgression[0].length, ARPEGGIO_DIVISIONS)
    Ts = arpeggio_generator(T*4, PROGRESSION_SIZE, PROGRESSION_SIZE*2)
    Ts = Ts if random.randint(0,1) == 1 else [T,T,T,T]
    
    # MELODY
    instrumentMelody = random.randint(1,5) + 144
    MELODY_DIVISIONS = 6
    times = arpeggio_generator(T*2, MELODY_DIVISIONS*2, ARPEGGIO_DIVISIONS)
    riffmelody = random.choices(scale, k = MELODY_DIVISIONS*4)
    playflag = [random.randint(0,1) for _ in range(MELODY_DIVISIONS*4)]

    # DRUMS
    bateria = random.randint(0,4)
    hh = random.randint(36,40)
    kick = random.randint(41,45)
    snare = random.randint(46,51)
    
    
    # AMBIENCE
    ambience = random.randint(36,49)

    
    # -------------------------- MUSIC STARTS-------------------------------#
    # AMBIENCE
    midiout.send_message([0x96, ambience, 100])
    
    # INTRO
    arpeggiointro = arpeggio if random.randint(0,1) == 1 else [0, 0, T]
    playHarmony(chordProgression, instrumentHarmony, arpeggiointro)
    
    # FIRST VERSE
    for i in range(0,4):
        arpeggiopv = arpeggio if random.randint(0,1) == 1 else [0, 0, T]
        t1 = threading.Thread(target=playHarmony, args=(chordProgression, instrumentHarmony, arpeggiopv))
        t2 = threading.Thread(target=playDrums, args=(chordProgression,bateria))
        t3 = threading.Thread(target=playBass, args=(chordProgression,))
        
        t1.start()
        t2.start()
        t3.start()

        t1.join()
        t2.join()
        t3.join()
    
    # PRIMEIRO CHORUS
    for i in range(0,2):
        arpeggiopr = arpeggio if random.randint(0,1) == 1 else [0, 0, T]
        t1 = threading.Thread(target=playHarmony, args=(chordProgression, instrumentHarmony, arpeggiopr))
        t2 = threading.Thread(target=playDrums, args=(chordProgression,bateria))
        t3 = threading.Thread(target=playMelody, args=(chordProgression, instrumentMelody, arpeggiopr, scale, riffmelody, playflag, times))
        t4 = threading.Thread(target=playBass, args=(chordProgression,))
        
        t1.start()
        t2.start()
        t3.start()
        t4.start()
        
        t1.join()
        t2.join()
        t3.join()
        t4.join()
    
    # SECOND VERSE
    for i in range(0,4):
        arpeggiosv = arpeggio if random.randint(0,1) == 1 else [0, 0, T]
        t1 = threading.Thread(target=playHarmony, args=(chordProgression, instrumentHarmony, arpeggiosv))
        t2 = threading.Thread(target=playDrums, args=(chordProgression,bateria))
        t3 = threading.Thread(target=playBass, args=(chordProgression,))
            
        t1.start()
        t2.start()
        t3.start()
        
        t1.join()
        t2.join()
        t3.join()

    # SECOND CHORUS
    for i in range(0,2):
        arpeggiosr = arpeggio if random.randint(0,1) == 1 else [0, 0, T]
        t1 = threading.Thread(target=playHarmony, args=(chordProgression, instrumentHarmony, arpeggiosr))
        t2 = threading.Thread(target=playDrums, args=(chordProgression,bateria))
        t3 = threading.Thread(target=playMelody, args=(chordProgression, instrumentMelody, arpeggiopr, scale, riffmelody, playflag, times))
        t4 = threading.Thread(target=playBass, args=(chordProgression,))
        
        t1.start()
        t2.start()
        t3.start()
        t4.start()
        
        t1.join()
        t2.join()
        t3.join()
        t4.join()
        
    #ACORDEFINAL
    playHarmony([chordProgression[0]], instrumentMelody, [0,0,T])
    
    time.sleep(2)

KeyboardInterrupt: 

In [22]:
def playHarmony(chordProgression, instrument, arpeggio):
    arpeggios = arpeggios_generator(arpeggio, Ts) 
    j = 0
    silence = chordProgression[0]
    for chord in chordProgression:
        if chord == silence and (j != 0):
            time.sleep(Ts[j])
        else:
            octaves = silence.octaves()
            for i, note in enumerate(silence.notes()):
                midiout.send_message([instrument, note2MIDI(note, octaves[i]), 0])
            octaves = chord.octaves()
            for i, note in enumerate(chord.notes()):
                midiout.send_message([instrument, note2MIDI(note, octaves[i]), 112])
                time.sleep(arpeggios[j][i])
        j = j+1
        silence = chord
    
    #silence all
    j = 0
    for chord in chordProgression:
        for i, note in enumerate(chord.notes()):
            midiout.send_message([instrument, note2MIDI(note, octaves[i]), 0])

In [43]:
playHarmony(chordProgression, instrumentHarmony, arpeggio)

KeyboardInterrupt: 

In [23]:
def playMelody(chordProgression, instrument, arpeggio, scale, riffmelody, playflag, times):
    j = 0
    for chord in chordProgression:
        midiout.send_message([instrument, note2MIDI(chord.notes()[0], 4), 112])
        for i in range(MELODY_DIVISIONS):
            if playflag[j] == 1:
                midiout.send_message([instrument, note2MIDI(riffmelody[j], 4), 112])
            time.sleep(times[j])
            j = (j+1)%(MELODY_DIVISIONS*2)
            
           


In [37]:
playMelody(chordProgression, instrumentMelody, arpeggio, scale)

TypeError: playMelody() missing 3 required positional arguments: 'riffmelody', 'playflag', and 'times'

In [24]:
def playBass(chordProgression):
    instrument = 144 + 7
    for chord in chordProgression:
        play = random.randint(0,1)
        if play == 1:
            midiout.send_message([instrument, chord.indexes()[0] + 36, 112])
        time.sleep(T)

In [25]:
def playDrums(chordProgression, bateria):
    instrument = 144
    drumflag = False
    for chord in chordProgression:
        if drumflag == False:
            drumflag = True
        else:
            drumflag = False
        playDrumsprimitivo(bateria, drumflag)


In [40]:
playDrums(chordProgression, bateria)

In [41]:
arpeggio = [element * 2 for element in arpeggio]
print(arpeggio)

[0.3, 2.1, 1.2000000000000002]


In [27]:
def arpeggios_generator(arpeggio, Ts):
    r = []
    for i in range(PROGRESSION_SIZE):
        r = r + [[j*Ts[i]/T for j in arpeggio]]
    return r